In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
import timm

timm.list_models('*tf_efficientnet_b7*')

['tf_efficientnet_b7']

In [6]:
from src.models.unet import Unet
import torch 

# unet = Unet('resnet18')
unet = Unet('tf_efficientnet_b7', in_chans=5, t=3, scale_factor=2)
unet;

In [7]:
inputs = torch.randn(10, 5, 256, 256)
features = unet.encoder(inputs)
for f in features:
    print(f.shape)

torch.Size([10, 32, 128, 128])
torch.Size([10, 48, 64, 64])
torch.Size([10, 80, 32, 32])
torch.Size([10, 224, 16, 16])
torch.Size([10, 640, 8, 8])


In [8]:
inputs = torch.randn(10, 256, 256, 3, 5)

output = unet(inputs) # B, H, W, T, C

output.shape

torch.Size([10, 1, 256, 256])

In [9]:
import torch 

traced = torch.jit.trace(unet, inputs)

traced;

/home/juan/.local/lib/python3.10/site-packages/einops/einops.py:316: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  known: Set[str] = {axis for axis in composite_axis if axis_name2known_length[axis] != _unknown_axis_length}
/home/juan/.local/lib/python3.10/site-packages/einops/einops.py:317: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  unknown: Set[str] = {axis for axis in composite_axis if axis_name2known_length[axis] == _unknown_axis_length}


In [10]:
traced.save('unet.pt')

In [11]:
loaded = torch.jit.load('unet.pt')
loaded;

loaded.eval()
loaded.cuda()
output = loaded(torch.randn(10, 256, 256, 3, 5).cuda())
output.shape

In [16]:
import torch 

scripted = torch.jit.script(unet)
scripted;

RuntimeError: 

forward(__torch__.src.models.decoder.___torch_mangle_1387.DecoderBlock self, Tensor x, Tensor skip) -> Tensor:
Expected a value of type 'Tensor (inferred)' for argument 'skip' but instead found type 'Optional[Tensor]'.
Inferred 'skip' to be of type 'Tensor' because it was not annotated with an explicit type.
:
  File "/home/juan/Desktop/competis/Contrails/src/models/decoder.py", line 124
        x = features[0]
        for i, decoder_block in enumerate(self.blocks):
            x = decoder_block(x, features[i+1] if i+1 < len(features) else None)
                ~~~~~~~~~~~~~ <--- HERE
        x = F.interpolate(x, scale_factor=2, mode='bilinear', align_corners=True)
        x = self.out_conv(x)


In [ ]:
scripted.save('unet.pt')

In [ ]:
loaded = torch.jit.load('unet.pt')
loaded;

In [ ]:
loaded.eval()
loaded.cuda()
output = loaded(torch.randn(1, 9, 256, 256).cuda())
output.shape

AttributeError: 'list' object has no attribute 'shape'

In [ ]:
for o in output:
	print(o.shape)

torch.Size([1, 64, 128, 128])
torch.Size([1, 64, 64, 64])
torch.Size([1, 128, 32, 32])
torch.Size([1, 256, 16, 16])
torch.Size([1, 512, 8, 8])
